In [1]:
import requests
import time, random
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [2]:
url = "http://vllm-serving-service:8888/v1/completions"
request_payload_file = '/data/Promptshett_codegen_50.txt'
# report_path = '/data/vLLM_Base_1gpu.csv'
df = pd.DataFrame(columns=['Prompt_No', 'Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [3]:
context_code_gen = """Generate a dbt model which gives me month-wise sum of payment done by each customer.
    {
    "schema": [
        {
            "Table Name": "customers",
            "Column Names": [
				"customer_id",
				"name",
				"email",
				"phone_number",
				"address",
				"registration_date"
            ]
        },
        {
            "Table Name": "orders",
            "Column Names": [
				"order_id",
				"customer_id",
				"order_date",
				"total_amount",
				"status"
            ]
        },
		{
            "Table Name": "payments",
            "Column Names": [
                "payment_id",
                "order_id",
                "payment_date",
                "amount",
                "payment_method"
            ]
        }
    ],
    "task_type": "code_generation"
}
{{ config(
    materialized = 'view'
) }}

WITH monthly_payments as (
        SELECT
          c.customer_id,
          DATE_TRUNC('month', p.payment_date) as payment_month,
          SUM(p.amount) as total_payment
        FROM {{ ref('stg_customers') }} c
        INNER JOIN {{ ref('stg_orders') }} o ON c.customer_id = o.customer_id
        INNER JOIN {{ ref('stg_payments') }} p ON o.order_id = p.order_id
        GROUP BY c.customer_id, DATE_TRUNC('month', p.payment_date)
    )
final as (	  
      SELECT
        customer_id,
        payment_month,
        total_payment
      FROM monthly_payments
	)
select * from final"""

question_cod_gen = """Generate a sql for dbt model which gives me project-wise details of employee who works for US-based clients.
"""

In [4]:
prompt =f"""You are an exceptionally intelligent DBT coding assistant that consistently delivers accurate and reliable responses to user instructions. Make sure you follow these rules:
               1. Use context to understand some relevant code examples for the given question.
               2. Use Schema provided in context section
               3. Generate only DBT code
               4. Do not use macros
               5. Provide explaination for the generated code
                @@ Instruction
 
                Context:
                {context_code_gen}
 
                Question:
                {question_cod_gen}
 
                @@ Response"""

In [5]:
len(prompt)

2139

In [16]:
# payload = {"prompt": "Problem: Create an animated dropdown menu in CSS that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu).", 
#            "max_tokens": 200,
#            "temperature": 0,
#            "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [7]:
payload = {"prompt": prompt, 
           "max_tokens": {},
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [8]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, a,b= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt_No':b,'Prompt':a,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result,a,b
    return calculate_execution_time

In [9]:
def save_report(file_path, df):
#     print("Saving report\n", df)
    df.to_csv(file_path, index=False)

In [10]:
@my_decorator
def inference(prompt_index, payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, json=payload_data)
#     print(f"Prompt {prompt_index}==> {payload_data['prompt']} \n {response.text}")
    print(f"Prompt {prompt_index}==> Response \n {response.text}")
    return response, payload_data['prompt'], prompt_index
    
        

In [11]:
# #Model Inference(Request) Same Payload
if __name__ == "__main__":
    for i in range(5):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
            futures = [executor.submit(inference, srno, payload) for srno in range(1)]
            concurrent.futures.wait(futures)
#     save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt 0==> Response 
 {"object":"error","message":"This model's maximum context length is 4096 tokens. However, you requested 4384 tokens (584 in the messages, 3800 in the completion). Please reduce the length of the messages or completion.","type":"BadRequestError","param":null,"code":400}
Execution time: 0.068971 seconds 


loop 1+++++++++++++++++++++++++++++
Prompt 0==> Response 
 {"object":"error","message":"This model's maximum context length is 4096 tokens. However, you requested 4384 tokens (584 in the messages, 3800 in the completion). Please reduce the length of the messages or completion.","type":"BadRequestError","param":null,"code":400}
Execution time: 0.067798 seconds 


loop 2+++++++++++++++++++++++++++++
Prompt 0==> Response 
 {"object":"error","message":"This model's maximum context length is 4096 tokens. However, you requested 4384 tokens (584 in the messages, 3800 in the completion). Please reduce the length of the messages or comp